In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LinearRegression
import json
import csv
from sklearn import metrics

In [2]:
with open('kernels.json') as json_file:
    kernels = json.load(json_file)

In [3]:
with open('squezzenet_layers.txt') as json_file:
    data = json.load(json_file)
    
    names = []
    for layer in data:
        names.append(layer)
        
    itens = []
    for name in names:
        itens.append(data[name])

In [4]:
    conv_2d = [[],[],[],[],[],[],[],[],[],[],[]]
    max_pol = [[],[],[],[],[],[],[],[],[],[],[]]
    
    conv_2d_y = []
    max_pol_y = []
    
    newData = {}
    i = 0

In [5]:
    for kernel,layer in zip(kernels,itens):
        
        if(kernel["kernel"] == "CONV_2D"):
            conv_2d[0].extend(np.repeat(layer["output"][0], len(kernel["averagePower"])))
            conv_2d[1].extend(np.repeat(layer["output"][1], len(kernel["averagePower"])))
            conv_2d[2].extend(np.repeat(layer["output"][2], len(kernel["averagePower"])))
            conv_2d[3].extend(np.repeat(layer["filters"][0], len(kernel["averagePower"])))
            conv_2d[4].extend(np.repeat(layer["filters"][1], len(kernel["averagePower"])))
            conv_2d[5].extend(np.repeat(layer["filters"][2], len(kernel["averagePower"])))
            conv_2d[6].extend(np.repeat(layer["stride"], len(kernel["averagePower"])))
            conv_2d[7].extend(np.repeat(layer["input"][0], len(kernel["averagePower"])))
            conv_2d[8].extend(np.repeat(layer["input"][1], len(kernel["averagePower"])))
            conv_2d[9].extend(np.repeat(layer["input"][2], len(kernel["averagePower"])))
#             newData[name]["executionTime"] = np.asarray(kernel["executionTime"])
            conv_2d[10].extend(np.asarray(kernel["averagePower"]))
            i = i + 1
            continue
        
#         if(kernel["kernel"] == "MAX_POOL_2D"):
            
#             max_pol[0].extend(np.repeat(layer["output"][0], len(kernel["averagePower"]))) 
#             max_pol[1].extend(np.repeat(layer["output"][1], len(kernel["averagePower"])))
#             max_pol[2].extend(np.repeat(layer["output"][2], len(kernel["averagePower"])))
#             max_pol[3].extend(np.repeat(layer["filters"][0], len(kernel["averagePower"])))
#             max_pol[4].extend(np.repeat(layer["filters"][1], len(kernel["averagePower"])))
#             max_pol[5].extend(np.repeat(layer["filters"][2], len(kernel["averagePower"])))
#             max_pol[6].extend(np.repeat(layer["stride"], len(kernel["averagePower"])))
#             max_pol[7].extend(np.repeat(layer["output"][0], len(kernel["averagePower"]))) 
#             max_pol[8].extend(np.repeat(layer["output"][1], len(kernel["averagePower"])))
#             max_pol[9].extend(np.repeat(layer["output"][2], len(kernel["averagePower"])))
# #             newData[name]["executionTime"] = np.asarray(kernel["executionTime"])
#             max_pol_y.extend(np.asarray(kernel["averagePower"]))
#             i = i + 1
#             continue

In [6]:
    training_conv = []
    evaluating_conv = []
    
    conv_2d = np.transpose(conv_2d)
    
    np.random.shuffle(conv_2d)
    
    conv_2d = np.transpose(conv_2d)

    percentage = 0.8
    length = round(percentage * len(conv_2d[0]))
    
    for column in conv_2d:
        training_conv.append(column[:length])
        evaluating_conv.append(column[length:])

    training_y = np.asarray(training_conv[10])
    evaluating_y = np.asarray(evaluating_conv[10])
    
    training_conv = np.asarray(training_conv)
    evaluating_conv = np.asarray(evaluating_conv)
    
    training_conv = training_conv[:-1]
    evaluating_conv = evaluating_conv[:-1]
    
    print(len(conv_2d[0]))

56160


In [7]:
    training_conv = np.transpose(training_conv)
    evaluating_conv = np.transpose(evaluating_conv)
    
    reg = LinearRegression()
    reg.fit(training_conv, training_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
    y_pred = reg.predict(evaluating_conv)

In [9]:
    print('Intercept: \n', reg.intercept_)
    print('Coefficients: \n', reg.coef_)

Intercept: 
 14436594.517744172
Coefficients: 
 [ 4.12748267e+09 -4.12761157e+09  1.43285102e+08 -1.43285102e+08
  3.76392265e+06 -3.76392296e+06 -1.44365899e+07  1.12502099e+07
 -1.11213118e+07 -8.89301300e-04]


In [10]:
    reg.score(evaluating_conv, evaluating_y)

0.5985495219941448

In [11]:
    print('Mean Absolute Error:', metrics.mean_absolute_error(evaluating_y, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(evaluating_y, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(evaluating_y, y_pred)))

Mean Absolute Error: 0.16605839260835253
Mean Squared Error: 0.06065486383721834
Root Mean Squared Error: 0.2462820818436013
